In [2]:
using JuMP, Gurobi
using CSV, LinearAlgebra, DataFrames
using Plots
using DelimitedFiles

In [3]:
path = "C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/src/deterministic/data/";

In [4]:
#load social and ecological data and orgnize 
FL = repeat([Float64], inner=15)
dtype = append!([String], FL);

regional_EF = CSV.File(string(path,"EF_SC31.csv"),header=1,delim=",", types=dtype) |> DataFrame    
capacity = CSV.File(string(path,"capacity31.csv"),header=1,delim=",", types=dtype) |> DataFrame    
distance = CSV.File(string(path,"distance31.csv"),header=1,delim=",") |> DataFrame 
LCA_model = CSV.File(string(path,"LCA_model.csv"), header=1, delim=",") |> DataFrame 
SDD = CSV.File(string(path,"SDD31.csv"),header=1,delim=",") |> DataFrame;

In [5]:
global_sink = 1.099e10                        # global pub (ocean) CO2 sequestration (ton/yr)
global_sink_tot = 2.236e10                  # global total (ocean+land) CO2 sequestration (ton/yr)
global_emi = 3.53e10                          # global CO2 emission (ton/yr)                          # 2021 global GD ($/yr)
es_ratio = global_sink/global_emi

emission_c = SDD[!, "emission"]          # national CO2 emission (ton/yr)
sink_c = SDD[!, "sink ton/yr"]           # national CO2 sink (ton/yr)
Dsoc = SDD[!, "Dsoc ton/yr"]           # national CO2 sink (ton/yr)

EF_trans = 1.005/10000                        # ton CO2/km*ton (The average freight truck in the U.S. emits 161.8 grams of CO2 per ton-mile)
process = LCA_model[!,"process"]
countries = capacity[!,"country"]
ncty = size(countries,1)                          # No. of countries
nproc = size(process,1);                          # No. of processes 

mkt_loc = findfirst(isequal("United States"), countries)
mkt_proc = findfirst(isequal("battery"), process)

# seperate model
cathode = collect(1:4)
cell = collect(5:10)
noncell = [12,13]
battery = [11,14]
scaler = LCA_model[!,"scaler"];

In [6]:
cell_demand = 500000;

---

In [75]:
function new_obj1(threshold, search=false, TL_cond=true)
    model = Model(Gurobi.Optimizer)
    set_optimizer_attributes(model, "NonConvex" => 2, "IntegralityFocus" => 1)
    set_optimizer_attribute(model, MOI.Silent(), true)
    slack = 0.00001

    @variable(model, x[1:ncty, 1:nproc] >= 0)
    @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
    @variable(model, tl[1:ncty] >= 0)

    x_cth = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,5] for i in 1:ncty) * scaler[k]) for k in cathode]
    x_cell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,11] for i in 1:ncty) * scaler[k]) for k in cell]
    x_noncell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,14] for i in 1:ncty) * scaler[k]) for k in noncell]
    x_battery = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,15] for i in 1:ncty) * scaler[k]) for k in battery];

    cstr_cap = [@constraint(model, x[i,k] <= capacity[!, 2:end][i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) == x[i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
    cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,11]*scaler[k]) for k in cell for j in 1:ncty]
    cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
    cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,15]*scaler[k]) for k in battery for j in 1:ncty]

    @constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) == cell_demand)
    tmp = filter!(e->e!=mkt_loc,collect(1:ncty))
    for j in tmp
        @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0);
    end

    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1)  # ncty*1 matrix
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)*ones(nproc,1)  # ncty*1 matrix


    transD = Vector{AffExpr}(undef, ncty)
    for j in 1:ncty
        arc_emi = 0
        arc_seq = 0
        for i in 1:ncty
            amount = sum(y[i,j,k] for k in 1:nproc)
            arc_emi += amount * distance[!, 2:end][i,j] * EF_trans
        end
        transD[j] = arc_emi  
    end

    Allo_soc = proD ./ emission_c .* Dsoc
    SS = transD + proD - Allo_soc

    for i in 1:ncty
        @constraint(model, SS[i] >= slack)
        @constraint(model, tl[i] * proS[i] == proD[i] + transD[i])
        if TL_cond == true
            @constraint(model, tl[i] <= threshold)
        end
    end

    EO = sum(proD + transD - proS)
    
    @objective(model, Min, EO)
    JuMP.optimize!(model)
    
    if search == true
        if termination_status(model) == MOI.OPTIMAL
            println(threshold)
        end
    else
        opt_x = JuMP.value.(x)
        opt_y = JuMP.value.(y)
        opt_obj = JuMP.objective_value(model);
        result = Dict(["x"=>opt_x, "obj"=>opt_obj, "y"=>opt_y])
        return result
    end
    

end


new_obj1 (generic function with 3 methods)

In [76]:
for threshold in 2.8:0.01:2.9
    new_obj1(threshold, true, true)
end

Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
2.88
Academic license - for non-commercial use only - expires 2024-12-26
2.89
Academic license - for non-commercial use only - expires 2024-12-26
2.9


In [81]:
res1 = new_obj1(2.88, false, true)

Academic license - for non-commercial use only - expires 2024-12-26


Dict{String, Any} with 3 entries:
  "obj" => 1.05664e6
  "x"   => [6200.0 0.0 … 0.0 0.0; 5579.65 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0;…
  "y"   => [0.0 0.0 … 0.0 6200.0; 0.0 0.0 … 0.0 5579.65; … ; 0.0 0.0 … 0.0 0.0;…

In [82]:
res0 = new_obj1(2.88, false, false)

Academic license - for non-commercial use only - expires 2024-12-26


Dict{String, Any} with 3 entries:
  "obj" => 1.0567e6
  "x"   => [6200.0 0.0 … 0.0 0.0; 5579.65 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0;…
  "y"   => [0.0 0.0 … 0.0 6200.0; 0.0 0.0 … 0.0 5579.65; … ; 0.0 0.0 … 0.0 0.0;…

---

In [84]:
pct_eo = 3.667e6
opt_eo = 1.05664e6;

In [106]:
ep = (pct_eo - opt_eo) / 4
eo_m = opt_eo + 1 * ep

1.70923e6

In [107]:
function pct_count(x,y)
    proD = x .* Matrix(regional_EF[:,2:end])
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = x .* pro_sink

    transD = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            emi = 0
            for j in 1:ncty
                emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans  
            end
            transD[i,k] = emi
        end
    end

    MEO = proD - proS + transD;
    
    indices = (MEO .>= 0.1) .|| (MEO .<= -0.1)
    ls = MEO[indices]
    total = length(ls)
    
    num_os = 0
    num_safe = 0
    for e in ls
        if e > 0
            num_os += 1
        elseif e < 0
            num_safe += 1
        end
    end
    
#     return proD, proS, transD

    return num_os,num_safe
end

pct_count (generic function with 1 method)

In [108]:
model = Model(Gurobi.Optimizer)
set_optimizer_attributes(model, "NonConvex" => 2, "IntegralityFocus" => 1)

@variable(model, x[1:ncty, 1:nproc] >= 0)
@variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
@variable(model, 0<=obj<=1)

@variable(model, delta[1:ncty, 1:nproc], Bin)
@variable(model, z[1:ncty, 1:nproc], Bin)
@variable(model, r[1:ncty, 1:nproc], Bin)
@variable(model, t[1:ncty, 1:nproc], Bin);

x_cth = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,5] for i in 1:ncty) * scaler[k]) for k in cathode]
x_cell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,11] for i in 1:ncty) * scaler[k]) for k in cell]
x_noncell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,14] for i in 1:ncty) * scaler[k]) for k in noncell]
x_battery = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,15] for i in 1:ncty) * scaler[k]) for k in battery];

cstr_cap = [@constraint(model, x[i,k] <= capacity[!, 2:end][i,k]) for k in 1:nproc for i in 1:ncty]
cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) == x[i,k]) for k in 1:nproc for i in 1:ncty]
cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,11]*scaler[k]) for k in cell for j in 1:ncty]
cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,15]*scaler[k]) for k in battery for j in 1:ncty]

@constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) == cell_demand)
tmp = filter!(e->e!=mkt_loc,collect(1:ncty))
for j in tmp
    @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0);
end

proD = x .* Matrix(regional_EF[:,2:end])
pro_sink = zeros(ncty, nproc)
for k in 1:nproc
    for i in 1:ncty
        pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
    end
end
proS = x .* pro_sink

transD = Matrix{AffExpr}(undef, ncty, nproc)
for k in 1:nproc
    for i in 1:ncty
        emi = 0
        for j in 1:ncty
            emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans  #PRODUCER
        end
        transD[i,k] = emi
    end
end

slack = 1e-6
M = 1e7

Allo_Dsoc = (proD*ones(nproc,1)) .* (Dsoc./emission_c)
SS = transD*ones(nproc,1) + proD*ones(nproc,1) - Allo_Dsoc
cstr_soc = [@constraint(model, SS[i] >= slack) for i in 1:ncty];

EO = proD - proS + transD
for i in 1:ncty
    for k in 1:nproc
        @constraint(model, EO[i,k] >= -M*r[i,k] + slack*t[i,k])
        @constraint(model, EO[i,k] <= -slack*r[i,k] + M*t[i,k])
        @constraint(model, z[i,k] + r[i,k] + t[i,k] == 1)
        @constraint(model, delta[i,k] => {x[i,k] >= 0.1})
    end
end

@constraint(model, obj*sum(delta) == sum(t))
@constraint(model, sum(EO) == eo_m);
    

Academic license - for non-commercial use only - expires 2024-12-26


In [ ]:
@objective(model, Min, obj)
JuMP.optimize!(model)

xx = JuMP.value.(x)
yy = JuMP.value.(y)
zz = JuMP.value.(z)
rr = JuMP.value.(r)
tt = JuMP.value.(t)
solution = JuMP.objective_value(model)


In [105]:
pct_count(xx,yy)

(8, 23)

In [100]:
8 / (8+23)

0.25806451612903225

---

### Min max

In [8]:
model = Model(Gurobi.Optimizer)
set_optimizer_attributes(model, "NonConvex" => 2, "IntegralityFocus" => 1)
# set_optimizer_attribute(model, MOI.Silent(), true)

@variable(model, x[1:ncty, 1:nproc] >= 0)
@variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
@variable(model, TL[1:ncty] >= 0)
@variable(model, z >= 0)

slack = 0.00001

x_cth = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,5] for i in 1:ncty) * scaler[k]) for k in cathode]
x_cell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,11] for i in 1:ncty) * scaler[k]) for k in cell]
x_noncell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,14] for i in 1:ncty) * scaler[k]) for k in noncell]
x_battery = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,15] for i in 1:ncty) * scaler[k]) for k in battery];

cstr_cap = [@constraint(model, x[i,k] <= capacity[!, 2:end][i,k]) for k in 1:nproc for i in 1:ncty]
cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) == x[i,k]) for k in 1:nproc for i in 1:ncty]
cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,11]*scaler[k]) for k in cell for j in 1:ncty]
cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,15]*scaler[k]) for k in battery for j in 1:ncty]

@constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) == cell_demand)
tmp = filter!(e->e!=mkt_loc,collect(1:ncty))
for j in tmp
    @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0);
end

proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1)  # ncty*1 matrix
pro_sink = zeros(ncty, nproc)
for k in 1:nproc
    for i in 1:ncty
        pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
    end
end
proS = (x.*pro_sink)*ones(nproc,1)  # ncty*1 matrix


transD = Vector{AffExpr}(undef, ncty)
for j in 1:ncty
    arc_emi = 0
    arc_seq = 0
    for i in 1:ncty
        amount = sum(y[i,j,k] for k in 1:nproc)
        arc_emi += amount * distance[!, 2:end][i,j] * EF_trans
    end
    transD[j] = arc_emi  
end

Allo_soc = proD ./ emission_c .* Dsoc
SS = transD + proD - Allo_soc

for i in 1:ncty
    @constraint(model, SS[i] >= slack)
    @constraint(model, TL[i] * proS[i] == proD[i] + transD[i])
    @constraint(model, z >= TL[i])
end

Academic license - for non-commercial use only - expires 2024-12-26


In [ ]:
@objective(model, Min, z)
JuMP.optimize!(model)

In [ ]:
opt_x = JuMP.value.(x)
opt_y = JuMP.value.(y)
opt_obj = JuMP.objective_value(model);
result = Dict(["x"=>opt_x, "obj"=>opt_obj, "y"=>opt_y])